In [3]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import requests

knn = NearestNeighbors()
td_idf_vectorizer = TfidfVectorizer()

## fdfd

Barselona data

In [2]:
listings_df_pd = pd.read_csv("listings.csv")
listings_df_pd_filtered = listings_df_pd[~listings_df_pd.name.isnull()].reset_index(drop=True)

train, test = train_test_split(listings_df_pd_filtered, test_size=0.2, random_state=42)
it_idf_names = td_idf_vectorizer.fit_transform(test.name)

knn.fit(X=it_idf_names)

five_closest = [list(knn.kneighbors(d,6)[1][0][1:] )for d in it_idf_names]
test['ground_truth'] = [test.iloc[d].id.to_list() for d in five_closest ]

test.to_parquet("train_test_data/test.pkt")
train.to_parquet("train_test_data/train.pkt")

wikidata

In [16]:
api_wiki = "https://wikimedia.org/api/rest_v1/metrics/pageviews/top/uk.wikisource/all-access/2019/04/all-days"
wiki = pd.DataFrame(requests.get(api_wiki).json()['items'][0]['articles'])

wiki['id'] = wiki.index
wiki_filtered = wiki[~wiki.Page.isnull()].reset_index(drop=True)

train, test = train_test_split(wiki_filtered, test_size=0.2, random_state=42)
it_idf_names = td_idf_vectorizer.fit_transform(test.Page)

knn.fit(X=it_idf_names)

five_closest = [list(knn.kneighbors(d,6)[1][0][1:] )for d in it_idf_names]
test['ground_truth'] = [test.iloc[d].id.to_list() for d in five_closest ]

test.to_parquet("train_test_data/wiki_test.pkt")
train.to_parquet("train_test_data/wiki_train.pkt")

<ipython-input-16-f2adfe88c784>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['ground_truth'] = [test.iloc[d].id.to_list() for d in five_closest ]


In [40]:
test

,Page,Edits,Editors,Views,id,ground_truth
575,Палеоліт,?,?,8625,575,"[305, 784, 773, 370, 516]"
259,Кличко Віталій Володимирович,?,?,12924,259,"[784, 305, 380, 516, 370]"
388,Члени речення,?,?,10440,388,"[552, 296, 669, 362, 331]"
495,Жири,?,?,9339,495,"[305, 784, 773, 516, 370]"
70,Театр корифеїв,0,0,22115,70,"[331, 669, 552, 362, 296]"
...,...,...,...,...,...,...
963,Біологічні ритми,?,?,6602,963,"[331, 669, 552, 362, 296]"
754,Конституційне право України,?,?,7519,754,"[741, 198, 436, 394, 305]"
774,Кров,?,?,7386,774,"[305, 784, 967, 370, 516]"
33,Клітина,0,0,33534,33,"[305, 784, 967, 370, 516]"
